## Qu'est ce que le clustering ? ##
=======================================

En data-science, le clustering est une technique de regroupement d'un ensemble de données en sous-ensembles ("clusters") de façon à ce que les données d'un même sous-ensemble soient plus similaires entre elles qu'avec celles des autres autres sous-ensembles. C'est une technique non-supervisée car elle ne nécessite pas de données de d'entraienement avec des résultats connus.



## Est ce un problème difficile ? pourquoi ? Donnez la complexité en temps et mémoire ##
=======================================

Oui, de mon point de vue, c'est un problème difficile :
- déterminer le nombre de clusters peut être compliqué lorsque l'on ne le connait pas à l'avance. On utilise la méthode du coude ou celle de la silhouette pour le déterminer au mieux mais cela peut être couteux en temps et en calculs.
- Optimiser le clustering (faire le tri au mieux) est très complexe => concept de problème NP-difficile
- la dimensionalité des données : plus il y a de dimensions, plus les calculs sont importants : mémoire et temps



## Quelle sont les métriques utilisés pour le clustering ? ##
=======================================

Coefficient de Silhouette, Somme des carrés intra-cluster, somme des carrés entre les clusters, indice de Davies-Bouldin, ARI, Pureté, F1-score



## 3 métriques avec ground truth (dont MNI) et 3 sans ground truth (dont silhouette) ##
=======================================

### Avec ground truth (avec connaissance terrain) ###
- ARI, Pureté, NMI (entre 0 et 1), AMI
### Sans ground truth (sans connaissance terrain) ###
- silhouette, WCSS, indice de Davies-Bouldin

NMI et AMI sont des métriques avec ground truth (comparaison des vrais clusters aux clusters prédits). Le coéficient silhouette évalue la qualité du clustering uniquement à partir des données, sans information extérieure.


=======================================


# 20_newgroups #

## K-means & TfidfVectorizer ##

In [6]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score, silhouette_score
from sklearn.metrics import classification_report


newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes')) # pour ne récupérer qu'une partie du dataset (pour me faciliter la vie et les temps de calculs)


In [11]:
vectorizer = TfidfVectorizer(stop_words='english') # instanciation  de l'algo de vectorisation, en supprimant les mots inutiles
vectors = vectorizer.fit_transform(newsgroups.data) # vectorisation des données

X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)

num_clusters = 20 # il y a 20 newsgroups
km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42) # clsutering
km.fit(X_train)

predicted_clusters = km.predict(X_test)

# Calculez les métriques
nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

# Affichez les résultats
print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

report = classification_report(y_test, predicted_clusters, target_names=newsgroups.target_names, zero_division=0)
print(report)

NMI: 0.32
AMI: 0.31
Silhouette Score: 0.01
                          precision    recall  f1-score   support

             alt.atheism       0.00      0.00      0.00       236
           comp.graphics       0.01      0.02      0.01       287
 comp.os.ms-windows.misc       0.00      0.00      0.00       290
comp.sys.ibm.pc.hardware       0.09      0.05      0.06       285
   comp.sys.mac.hardware       0.00      0.00      0.00       312
          comp.windows.x       0.03      0.03      0.03       308
            misc.forsale       0.03      0.27      0.06       276
               rec.autos       0.00      0.00      0.00       304
         rec.motorcycles       0.00      0.00      0.00       279
      rec.sport.baseball       0.00      0.00      0.00       308
        rec.sport.hockey       0.01      0.01      0.01       309
               sci.crypt       0.00      0.00      0.00       290
         sci.electronics       0.00      0.00      0.00       304
                 sci.med       1

## K-means & CountVectorizer ##

In [8]:
# 2ème façon
vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups.data)


X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)


num_clusters = 20


km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
km.fit(X_train)


predicted_clusters = km.predict(X_test)


nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

report = classification_report(y_test, predicted_clusters, target_names=newsgroups.target_names, zero_division=0)
print(report)


NMI: 0.01
AMI: 0.00
Silhouette Score: 0.91
                          precision    recall  f1-score   support

             alt.atheism       0.00      0.00      0.00       236
           comp.graphics       0.00      0.00      0.00       287
 comp.os.ms-windows.misc       1.00      0.00      0.01       290
comp.sys.ibm.pc.hardware       0.00      0.00      0.00       285
   comp.sys.mac.hardware       0.00      0.00      0.00       312
          comp.windows.x       0.00      0.00      0.00       308
            misc.forsale       0.05      1.00      0.09       276
               rec.autos       0.00      0.00      0.00       304
         rec.motorcycles       0.00      0.00      0.00       279
      rec.sport.baseball       0.00      0.00      0.00       308
        rec.sport.hockey       0.00      0.00      0.00       309
               sci.crypt       0.00      0.00      0.00       290
         sci.electronics       0.00      0.00      0.00       304
                 sci.med       0

## Constatations ##

Les silhouette Score sont complètement opposés. NMI et AMI diffèrent d'une vectorisation à une autre mais ne sont jamais concluants.

## LDA & TfidfVectorizer ##

In [9]:
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups.data)


X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)


num_topics = 20


lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X_train)


document_topics = lda.transform(X_test)
predicted_clusters = document_topics.argmax(axis=1)


nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

NMI: 0.02
AMI: 0.00
Silhouette Score: -0.24


## LDA & CountVectorizer ##

In [10]:
from sklearn.metrics import classification_report

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups.data)


X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)


num_topics = 20


lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X_train)


document_topics = lda.transform(X_test)
predicted_clusters = document_topics.argmax(axis=1)


nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

report = classification_report(y_test, predicted_clusters, target_names=newsgroups.target_names, zero_division=0)
print(report)


NMI: 0.32
AMI: 0.31
Silhouette Score: -0.27
                          precision    recall  f1-score   support

             alt.atheism       0.00      0.02      0.01       236
           comp.graphics       0.00      0.00      0.00       287
 comp.os.ms-windows.misc       1.00      0.02      0.03       290
comp.sys.ibm.pc.hardware       0.00      0.00      0.00       285
   comp.sys.mac.hardware       0.01      0.03      0.01       312
          comp.windows.x       0.00      0.00      0.00       308
            misc.forsale       0.03      0.00      0.01       276
               rec.autos       0.23      0.62      0.33       304
         rec.motorcycles       0.00      0.00      0.00       279
      rec.sport.baseball       0.00      0.00      0.00       308
        rec.sport.hockey       0.12      0.01      0.01       309
               sci.crypt       0.02      0.09      0.04       290
         sci.electronics       0.08      0.01      0.01       304
                 sci.med       

## Constatations ##

Aucun score n'est concluant. Je suis paumé ;-|

In [15]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np

newsgroups = fetch_20newsgroups(subset='all')  


labels = newsgroups.target
group_names = newsgroups.target_names
unique_labels, counts = np.unique(labels, return_counts=True)


for label, count in zip(unique_labels, counts):
    print(f"Group: {group_names[label]}, Number of messages: {count}")

Group: alt.atheism, Number of messages: 799
Group: comp.graphics, Number of messages: 973
Group: comp.os.ms-windows.misc, Number of messages: 985
Group: comp.sys.ibm.pc.hardware, Number of messages: 982
Group: comp.sys.mac.hardware, Number of messages: 963
Group: comp.windows.x, Number of messages: 988
Group: misc.forsale, Number of messages: 975
Group: rec.autos, Number of messages: 990
Group: rec.motorcycles, Number of messages: 996
Group: rec.sport.baseball, Number of messages: 994
Group: rec.sport.hockey, Number of messages: 999
Group: sci.crypt, Number of messages: 991
Group: sci.electronics, Number of messages: 984
Group: sci.med, Number of messages: 990
Group: sci.space, Number of messages: 987
Group: soc.religion.christian, Number of messages: 997
Group: talk.politics.guns, Number of messages: 910
Group: talk.politics.mideast, Number of messages: 940
Group: talk.politics.misc, Number of messages: 775
Group: talk.religion.misc, Number of messages: 628
